In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape
from tqdm import tqdm
import datetime as dt
import os
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
def scrape_tweets(query, max_tweets=-1,output_path=None ): 
    
    
    tweets_list = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if max_tweets != -1 and i >= int(max_tweets):
            break
        tweets_list.append([tweet.date, tweet.id, tweet.content])
    df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text'])
    if type(output_path)==str:
      output_path = os.path.join(output_path,dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+"-"+str(query)+".csv")
      df.to_csv(output_path, index=False)
      print("Data tweet tersimpan di",output_path)
    return df


In [ ]:
df = scrape_tweets("tes",5)
df

,Datetime,Tweet Id,Text
0,2022-11-28 15:07:14+00:00,1597245738551242755,@NvSafia Ils peuvent te demander de poser tes ...
1,2022-11-28 15:07:12+00:00,1597245733723607040,@Doud1620 @TFoot__ Vous etes con ou quoi il a ...
2,2022-11-28 15:07:11+00:00,1597245729604390913,@greenbvll @Back2021R @EcrisioFX Si ça mets ma...
3,2022-11-28 15:07:03+00:00,1597245695916126208,@BalouRetour @PinttowGoat @ActuFoot_ sa ce voi...
4,2022-11-28 15:07:00+00:00,1597245680480792576,Highest graded TEs from Week 13⭐️ https://t.co...


### Scraping per kandidat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Kumpulan Tugas/Rekdat Akhir'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Kumpulan Tugas/Rekdat Akhir


In [ ]:
candidates_df = pd.read_csv('candidates.csv')

candidates_df = candidates_df[candidates_df['party'].apply(lambda x : x in ["D","R"])] #Filter hanya calon republikan dan demokrat saja

candidates_df

,name,party,votes,isWinner,isIncumbent,district
0,Mary Peltola,D,136893,True,True,AK00
1,Sarah Palin,R,112255,False,False,AK00
2,Nick Begich,R,0,False,False,AK00
5,Jerry Carl,R,139854,True,True,AL01
7,Robert Aderholt,R,164371,True,True,AL04
...,...,...,...,...,...,...
1049,Lacy Watson,D,65428,False,False,WV01
1051,Alexander Mooney,R,160493,True,True,WV02
1052,Barry Wendell,D,84278,False,False,WV02
1053,Harriet Hageman,R,132215,True,False,WY00


In [ ]:
all_tweet_df = pd.DataFrame(columns=['Datetime', 'Tweet Id', 'Text','Candidate Name','Candidate Party','Candidate District'])
for index,row in tqdm(candidates_df.iterrows()):
  tweet_calon_ini_df = scrape_tweets(row['name'],max_tweets=200)
  tweet_calon_ini_df['Candidate Name'] = row['name']
  tweet_calon_ini_df['Candidate Party'] = row['party']
  tweet_calon_ini_df['Candidate District'] = row['district']
  all_tweet_df = pd.concat([all_tweet_df,tweet_calon_ini_df])

850it [1:15:14,  5.31s/it]


In [ ]:
all_tweet_df.to_csv('tweet_200percalon.csv',index=False)